In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans #The k-means algorithm

First, let's start by creating a user-item matrix, as explained in the other Notebook.

In [3]:
movie_file = pd.read_csv('movies.csv')
ratings_file = pd.read_csv('ratings.csv')
df = pd.merge(movie_file, ratings_file)

ratings = pd.pivot_table(df, index='userId', columns='title', values='rating')
ratings.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#This piece of code is a bit complex. Here it is, step by step:
#1. reindex shuffles a dataframe according to a new list
#2. ratings.count() gets the number of non-NaN values per column/movie
#3. sort_values() sort those values, descending (because ascending=False)
#4. finally, .index gets the names of the columns/movies
#axis=1 tells Pandas we want to reshuffle the columns (not the rows)
ratings = ratings.reindex(ratings.count().sort_values(ascending=False).index, axis=1)
ratings.head(3)

title,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),Schindler's List (1993),...,Shrooms (2007),Siam Sunset (1999),Side by Side (2012),Sightseers (2012),"Signal, The (2007)",Shot Caller (2017),"Signal, The (2014)",Silent Hill: Revelation 3D (2012),Silent Movie (1976),'71 (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,3.0,4.0,5.0,5.0,4.0,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
fav_movie = "Toy Story (1995)"
corr = ratings.corrwith(ratings[fav_movie])
corr.sort_values(ascending=False)[0:10]

/Users/Kolien/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3175: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/Kolien/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3109: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


title
O (2001)                         1.0
Last Dragon, The (1985)          1.0
Steel (1997)                     1.0
Left Behind: The Movie (2000)    1.0
Little Monsters (1989)           1.0
Amen. (2002)                     1.0
Chasing Liberty (2004)           1.0
Wolfman, The (2010)              1.0
Everybody Wants Some (2016)      1.0
All-Star Superman (2011)         1.0
dtype: float64

In [16]:
ratings_full = ratings.fillna(0) #fill the NaN with the mean of each column
ratings_full.head(3)

title,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),Schindler's List (1993),...,Shrooms (2007),Siam Sunset (1999),Side by Side (2012),Sightseers (2012),"Signal, The (2007)",Shot Caller (2017),"Signal, The (2014)",Silent Hill: Revelation 3D (2012),Silent Movie (1976),'71 (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,3.0,4.0,5.0,5.0,4.0,4.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
ratings_full.corrwith(ratings_full["Toy Story (1995)"], axis=0, drop=False).sort_values(ascending=False).head(10)

title
Toy Story (1995)                              1.000000
Toy Story 2 (1999)                            0.461761
Groundhog Day (1993)                          0.361540
Independence Day (a.k.a. ID4) (1996)          0.358473
Willy Wonka & the Chocolate Factory (1971)    0.357314
Mission: Impossible (1996)                    0.352847
Nutty Professor, The (1996)                   0.350295
Bug's Life, A (1998)                          0.345431
Lion King, The (1994)                         0.344248
Babe (1995)                                   0.341136
dtype: float64